We will now train a decision tree and a simple neural network, a multi-layer perceptron, to classify handwritten digits in the MNIST dataset

Below we load our libraries we'll be using and download the MNIST dataset

In [ ]:
# Read in the mnist digit dataset

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
import random
from sklearn import tree
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neural_network import MLPClassifier

# X is a pandas Dataframe with 70,000 rows and 784 columns. Each row represents
# one 28x28 image of a digit, with each column containing one pixel of the image.
# Pixel value ranges from 0.0 (background) to 255.0. y is a pandas Series with
# 70,000 categorical values ranging from 0 to 9, representing digit '0' to '9'.
# By removing return_X_y=True, fetch_openml would return an object of type
# sklearn.utils.Bunch instead. The object has data and target attributes,
# containing X and y values, respectively. It also has a DESCR attribute,
# with description of the dataset. Two additional attributes, feature_names and
# target_names, are not very useful for this dataset.
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

Next, we will divide the data into a training set and test set, randomly selecting 5000 examples for training.

In [ ]:
train_samples = 5000

# X is in pandas format for some reason. Convert to numpy.
X = np.array(X)
y = np.array(y)

# The original code randomly shuffles the data before calling train_test_split.
# This is unnecessary as train_test_split does random selection of the data,
# so I remove the shuffle. The original code doesn't pass a random state to
# train_test_split, so different train/test sets are returned each time. I added
# a random state with fixed seed, to make the selection repeatable.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=1000,
    random_state=check_random_state(0))

Let's print out the 417$^{\text{th}}$ item in the dataset and its label.

In [ ]:
i = 417
img = np.array(X_train[i]).reshape(28,28)
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.show()
y_train[i]

Let's see how a decision tree with 170 decision rules performs by training it and printing its accuracy.

In [ ]:
clf = tree.DecisionTreeClassifier(max_leaf_nodes = 170)	
clf = clf.fit(X_train, y_train)			
correct = 0						
for i in range(len(X_test)):	
  if clf.predict([X_test[i]]) == y_test[i]: correct = correct + 1
  acc = [100.0* correct / len(X_test)]
acc

Now let's try a simple neural network, a multi-layer perceptron with no hidden layers.

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=[], max_iter = 10000, activation = 'identity', random_state=check_random_state(0))
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(score)

Now, we will add one hidden layer and expand the number of hidden units from 10 to 200 in intervals of 10. We'll print the accuracy of each model given the number of hidden units.

In [ ]:
for i in range(1,21):
  nhidden = i*10
  clf = MLPClassifier(hidden_layer_sizes=[nhidden], max_iter = 10000, random_state=check_random_state(0))
  clf.fit(X_train, y_train)
  score = clf.score(X_test, y_test)
  print(nhidden, score)

To see how a model varies across multiple runs, we check it ten times.

In [ ]:
nhidden = 170
random_state=check_random_state(0)
for i in range(10):
  clf = MLPClassifier(hidden_layer_sizes=[nhidden], max_iter = 10000, random_state=random_state)
  clf.fit(X_train, y_train)
  score = clf.score(X_test, y_test)
  print(nhidden,score)

In [ ]:
# Take the best classifier we saw above, and show its errors.
# Some of the error cases are hard, even for human eyes, but some are easy for human.
nhidden = 180
clf = MLPClassifier(hidden_layer_sizes=[nhidden], max_iter = 10000, random_state=check_random_state(0))
clf.fit(X_train, y_train)
predicts = clf.predict(X_test)
for X, y, predict in zip(X_test, y_test, predicts):
  if predict != y:
    img = np.array(X).reshape(28,28)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.show()
    print('Predict:', predict, 'Actual:', y)